In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pickle
from pdf2image import convert_from_path
import tempfile

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm

import segment_boards

%matplotlib inline
def sbw(im):
  plt.imshow(im, cmap='gray', vmin=0, vmax=255)

def sw(im):
  plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
  plt.show()

In [5]:
model = pickle.load(open('..\\data\\dev\\set\\linear_des8.pkl', 'rb'))

In [6]:
%%time
folder = '..\\data\\out\\1n_final'
piecesnames = ['BlackBishop', 'BlackKing', 'BlackKnight', 'BlackPawn', 'BlackQueen', 'BlackRook', 'BlackSpace', 'WhiteBishop', 'WhiteKing', 'WhiteKnight', 'WhitePawn', 'WhiteQueen', 'WhiteRook', 'WhiteSpace']
label_len = len(piecesnames)
confusion_matrix = np.zeros((label_len, label_len), dtype=np.int32)

winSize = (128, 128) #
# winSize = (96, 96) #
blockSize = (64, 64) #
blockStride = (32, 32) #
cellSize = (16, 16) #

# blockSize = (16, 16) #
# blockStride = (8, 8) #
# cellSize = (8, 8) #
nbins = 9 #
derivAperture = 1
winSigma = -1.
histogramNormType = 0
L2HysThreshold = 0.2
gammaCorrection = 1
nlevels = 64
signedGradients = True #

hog = cv2.HOGDescriptor(
  winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma, histogramNormType,
  L2HysThreshold, gammaCorrection, nlevels, signedGradients
)

k = 0
for i, piece in enumerate(sorted(os.listdir(folder))):
  folder2 = os.path.join(folder, piece)
  if os.path.isfile(folder2):
    continue
  for j, filename in enumerate(sorted(os.listdir(folder2))):
    fullname = os.path.join(folder2, filename)
    im = cv2.imread(fullname, 0)
    im = cv2.resize(im, (128, 128))
#     im = cv2.resize(im, (125, 125))

    x = hog.compute(im, padding=(3,3))
#     x = hog.compute(im)
    y = int(model.predict(x.T))
    
    confusion_matrix[i][y] += 1
    if k % 1000 == 0:
      print(k)
    k += 1

expected = 26816
total = np.sum(confusion_matrix)
print(confusion_matrix)
print('accuracy', np.trace(confusion_matrix) / total)
print('total', total)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
[[ 589    0    0    0    0    0    0    0    0    0    0    1    0    0]
 [  83  332    0    0    0    0    0    4    0    0    0    0    0    0]
 [   0    0  538    1    0    0    0    0    0    2    0    0    0    0]
 [  23    0   16 2548    0    0    0    0    0    1    0    0    0    0]
 [   0    0    0    0  325    0    0    1    0    0    0   72    0    0]
 [   4    0    0    0    0  746    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0 8275    0    0    0    0    0    0    0]
 [   6    0    0    0    0    0    0  552    1    0    0    0    1    0]
 [   0    0    0    0    0    0    0    9  407    0    0    1    1    0]
 [   2    0    0    0    0    0    0    0    0  526    3    0    0    0]
 [   4    0    0    0   14    0    0    5    0   35 2535    3    0    0]
 [   0    0    0    0    0    0    0    0    0  

In [13]:
np.set_printoptions(suppress=True)

In [7]:
confusion_matrix.diagonal() / confusion_matrix.sum(1)

array([0.99830508, 0.79236277, 0.99445471, 0.98454405, 0.81658291,
       0.99466667, 1.        , 0.98571429, 0.97368421, 0.9905838 ,
       0.97650231, 1.        , 0.98807947, 1.        ])